#Downloading and Preprocessing the dataset

In [0]:
!wget http://www.itl.nist.gov/iaui/vip/cs_links/EMNIST/gzip.zip
!unzip gzip.zip
!rm gzip.zip

In [0]:
!pip install python-mnist

In [0]:
import numpy as np
import matplotlib.pyplot as plt
from mnist import MNIST
import tensorflow as tf
from tensorflow.keras import layers
from sklearn.utils.class_weight import compute_class_weight

In [6]:
#Load the entire dataset as numpy arrays
emnist_data = MNIST(path='gzip',return_type = 'numpy')
emnist_data.select_emnist('byclass')

x_train,y_train = emnist_data.load_training()
x_test,y_test = emnist_data.load_testing()

print(x_train.shape,y_train.shape)
print(x_test.shape,y_test.shape)

(697932, 784) (697932,)
(116323, 784) (116323,)


In [7]:
img_size = 28

x_train = x_train.reshape(-1,img_size,img_size,1).astype('float32')/255.0
x_test = x_test.reshape(-1,img_size,img_size,1).astype('float32')/255.0

print(x_train.shape)

(697932, 28, 28, 1)


In [8]:
num_classes = len(np.unique(y_train))
print("Number of classes "+str(num_classes))

unique_classes = np.unique(y_train)
class_weights = dict(enumerate(compute_class_weight('balanced', unique_classes,y_train)))

y_train = tf.keras.utils.to_categorical(y_train,num_classes=num_classes)
y_test = tf.keras.utils.to_categorical(y_test,num_classes=num_classes)

print(y_train.shape)

Number of classes 62
(697932, 62)


#Creating and Training the Model

In [0]:
def create_model():
  model = tf.keras.Sequential()
  model.add(layers.Conv2D(16,kernel_size=(3,3),padding='same',activation='relu',input_shape=(28,28,1)))
  model.add(layers.MaxPooling2D(pool_size=(2,2)))
  model.add(layers.BatchNormalization())
  model.add(layers.Dropout(0.4))

  model.add(layers.Conv2D(32,kernel_size=(3,3),padding='same',activation='relu'))
  model.add(layers.MaxPooling2D(pool_size=(2,2)))
  model.add(layers.BatchNormalization())
  model.add(layers.Dropout(0.4))

  model.add(layers.Conv2D(64,kernel_size=(3,3),padding='same',activation='relu'))
  model.add(layers.MaxPooling2D(pool_size=(2,2)))
  model.add(layers.BatchNormalization())
  model.add(layers.Dropout(0.4))

  model.add(layers.Flatten())
  model.add(layers.Dense(128,activation='relu'))

  model.add(layers.Dense(num_classes,activation='softmax'))

  return model

In [0]:
model = create_model()
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [28]:
es = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    mode = 'min',
    verbose=1,
    patience = 10,
    restore_best_weights=True
)

history = model.fit(x_train,y_train,batch_size=1024,epochs=200,verbose=1,callbacks=[es],
          validation_data=(x_test,y_test),shuffle=True)

682/682 [==============================] - 9s 14ms/step - loss: 0.3490 - accuracy: 0.8686 - val_loss: 0.3328 - val_accuracy: 0.8751
Epoch 39/200
682/682 [==============================] - 9s 14ms/step - loss: 0.3497 - accuracy: 0.8690 - val_loss: 0.3301 - val_accuracy: 0.8747
Epoch 40/200
682/682 [==============================] - 9s 14ms/step - loss: 0.3494 - accuracy: 0.8690 - val_loss: 0.3296 - val_accuracy: 0.8758
Epoch 41/200
682/682 [==============================] - 9s 14ms/step - loss: 0.3496 - accuracy: 0.8686 - val_loss: 0.3311 - val_accuracy: 0.8749
Epoch 00041: early stopping


In [29]:
score = model.evaluate(x_test,y_test,verbose=1)
print("Test Loss:{} and Test accuracy:{}".format(score[0],score[1]))

3636/3636 [==============================] - 8s 2ms/step - loss: 0.3282 - accuracy: 0.8770
Test Loss:0.3282179832458496 and Test accuracy:0.8769890666007996


In [0]:
def plot_history(x,metric):
    plt.plot(history.history[metric])
    plt.plot(history.history['val_'+metric])
    plt.title(metric)
    plt.xlabel("epoch")
    plt.ylabel(metric)
    plt.legend(['Train','Validation'])
    plt.show()

In [0]:
plot_history(model.history.history,'accuracy')
plot_history(model.history.history,'loss')

#Converting Model to JavaScript

In [0]:
model.save("cnn_emnist.h5")

In [0]:
!pip install tensorflowjs
!tensorflowjs_converter --input_format keras "cnn_emnist.h5" ./jsmodel